In [14]:
# readin data
win_dir = "..\data\\"
unix_dir = "../data/"
if(isdir(win_dir))
    data_dir = win_dir
elseif(isdir(unix_dir))
    data_dir = unix_dir
end
print(data_dir)

#cost of transhipment over boat. Takes one month
ocean_costs = readdlm(data_dir*"ocean_cost.tsv", '\t')
temp = zeros()
#cost of transhipment over air. Instant
air_costs = readdlm(data_dir*"air_cost.tsv", '\t')

#costs to hold a boat at non retail location
holding_costs = readdlm(data_dir*"holding_cost.tsv",'\t')
risk_costs = readdlm(data_dir*"risk_cost.tsv", '\t')
demand_7536 = readdlm(data_dir*"demand_7536.tsv",'\t')
demand_7878 = readdlm(data_dir*"demand_7878.tsv",'\t')
demand_7585 = readdlm(data_dir*"demand_7585.tsv",'\t')


../data/

12×6 Array{Float64,2}:
 110.0  168.0   90.0   86.0  50.0  45.0
 131.5  174.0  130.0   90.0  68.0  60.0
 169.5  195.0  175.0   96.0  70.0  60.0
 195.0  213.0  195.0  106.0  70.0  60.0
 180.0  235.0  178.0  120.0  87.0  75.0
 165.0  260.0  162.0  130.0  90.0  70.0
 155.0  241.0  152.0  124.0  78.0  65.0
 145.0  210.0  146.0  105.0  77.0  65.0
 132.5  189.0  130.0   94.0  60.0  65.0
 138.5  165.0  136.0   82.0  60.0  65.0
 122.5  160.0  120.0   80.0  42.0  50.0
 102.5  158.0  100.0   80.0  40.0  50.0

In [41]:
#Scaling of cost from undersupplying demand
lambda = 100

#net profit per boat at each retailer
net_profit_7536 = 1000*ones(6)
net_profit_7878 = 100*ones(6)
net_profit_7585 = 100*ones(6)

cur_product = "7536"

"7536"

In [42]:
# Select current profit/boat at each retailer
if cur_product == "7536"
    net_profit = net_profit_7536
    demand = demand_7536
elseif cur_product == "7878"
    net_profit = net_profit_7878
    demand = demand_7878
elseif cur_product == "7585"
    net_profit = net_profit_7585
    demand = demand_7585
end

#define an inverse incidence tensor for our flow (0 implies incidence)
#inv_incidence_tensor[t1,n1,t2,n2] = 0 implies
#node 1 at time t1 may send product to node 2 at time t2
#Note: There are 1:15 real nodes that represent locations,
#and 16:21 dummy nodes used to model undersupplying demand
inv_incidence_tensor = ones(12,21,12,21)

#cost of flows
cost_tensor = zeros(12,21,12,21)

#Expedite edges
for t in 1:12
    #1 can expedite to its hubs
    inv_incidence_tensor[t,1,t,2:3] = 0
    cost_tensor[t,1,t,2:3] = air_costs[1,2:3]
    
    
    #2 can expedite to its dc's
    inv_incidence_tensor[t,2,t,4:6] = 0
    cost_tensor[t,2,t,4:6] = air_costs[2,4:6]
    
    #3 can expedite to its dc's
    inv_incidence_tensor[t,3,t,7:9] = 0
    cost_tensor[t,3,t,7:9] = air_costs[3,7:9]
    
    #Some Dc's can expedite to eachother
    inv_incidence_tensor[t,4,t,5] = 0
    cost_tensor[t,4,t,5] = air_costs[4,5]
    inv_incidence_tensor[t,5,t,4] = 0
    cost_tensor[t,5,t,4] = air_costs[5,4]
    inv_incidence_tensor[t,7,t,8] = 0
    cost_tensor[t,7,t,8] = air_costs[7,8]
    inv_incidence_tensor[t,8,t,7] = 0
    cost_tensor[t,8,t,7] = air_costs[8,7]
    inv_incidence_tensor[t,8,t,9] = 0
    cost_tensor[t,8,t,9] = air_costs[8,9]
    inv_incidence_tensor[t,9,t,8] = 0
    cost_tensor[t,9,t,8] = air_costs[9,8]
    
    #Dc's can expedite to retailers
    for i in 4:9 
        inv_incidence_tensor[t,i,t,i+6] = 0
        cost_tensor[t,i,t,i+6] = air_costs[i,i+6] - net_profit[i-3]
    end
end

#Shipping edges
for t in 1:11
    #1 can ship to its hubs
    inv_incidence_tensor[t,1,t+1,2:3] = 0
    cost_tensor[t,1,t+1,2:3] = ocean_costs[1,2:3]
    
    
    #2 can ship to its dc's
    inv_incidence_tensor[t,2,t+1,4:6] = 0
    cost_tensor[t,2,t+1,4:6] = ocean_costs[2,4:6]
    
    #3 can ship to its dc's
    inv_incidence_tensor[t,3,t+1,7:9] = 0
    cost_tensor[t,3,t+1,7:9] = ocean_costs[3,7:9]
    
    #Some Dc's can ship to eachother
    inv_incidence_tensor[t,4,t+1,5] = 0
    cost_tensor[t,4,t+1,5] = ocean_costs[4,5]
    inv_incidence_tensor[t,5,t+1,4] = 0
    cost_tensor[t,5,t+1,4] = ocean_costs[5,4]
    inv_incidence_tensor[t,7,t+1,8] = 0
    cost_tensor[t,7,t+1,8] = ocean_costs[7,8]
    inv_incidence_tensor[t,8,t+1,7] = 0
    cost_tensor[t,8,t+1,7] = ocean_costs[8,7]
    inv_incidence_tensor[t,8,t+1,9] = 0
    cost_tensor[t,8,t+1,9] = ocean_costs[8,9]
    inv_incidence_tensor[t,9,t+1,8] = 0
    cost_tensor[t,9,t+1,8] = ocean_costs[9,8]
    
    #Dc's can ship to retailers
    for i in 4:9 
        inv_incidence_tensor[t,i,t+1,i+6] = 0
        cost_tensor[t,i,t+1,i+6] = ocean_costs[i,i+6] - net_profit[i-3]
    end
end

#Holding edges
for t in 1:11
    for n in 1:9
        inv_incidence_tensor[t,n,t+1,n] = 0
        cost_tensor[t,n,t+1,n] = holding_costs[n]
    end
end

#Dummy underDemand nodes (16:21). Used to represent when we under supply demand to retailers
for t in 1:12
    for n in 10:15
        inv_incidence_tensor[t,n+6,t,n] = 0
        cost_tensor[t,n+6,t,n] = lambda * risk_costs[n-9]
    end
end



In [43]:
using JuMP, Clp
#model
m = Model(solver =ClpSolver())

#Flow tensor
@variable(m, flow[1:12,1:21,1:12,1:21] >= 0)

#only allow edges defined by inv_incidence_tensor
@constraint(m, sum(flow .* inv_incidence_tensor) == 0)

#require conservation of flow for nodes 2:9
for n in 2:9
    for t in 1:12
        @constraint(m,sum(flow[:,:,t,n]) - sum(flow[t,n,:,:]) == 0)
    end
end

#require demands to be made by true supply and dummy nodes
for n in 10:15
    for t in 1:12
        @constraint(m, sum(flow[:,:,t,n]) == demand[t,n-9])
    end
end

#maximize profit by minimizing cost
@objective(m, Min, sum(flow.* cost_tensor))
solve(m)


:Optimal

In [46]:
println("Optimal solution achieves \$", -getobjectivevalue(m), " in profit.")

Optimal solution achieves $6.8924168e6 in profit.


In [50]:
flow_val = getvalue(flow)
println("Total boats sold: ", sum(flow_val[:,1,:,:]))
for t in 1:12
    println("Boats sold in month ", t, ": ", sum(flow_val[:,:,t,10:15]))
end

Total boats sold: 12132.0
Boats sold in month 1: 846.0
Boats sold in month 2: 949.0
Boats sold in month 3: 1101.0
Boats sold in month 4: 1159.0
Boats sold in month 5: 1206.0
Boats sold in month 6: 1203.0
Boats sold in month 7: 1093.0
Boats sold in month 8: 1060.0
Boats sold in month 9: 943.0
Boats sold in month 10: 874.0
Boats sold in month 11: 885.0
Boats sold in month 12: 813.0


In [52]:
for t in 1:11
    total = 0
    for n in 1:9
        total += flow_val[t,n,t+1,n]
    end
    println("Boats held after month ",t, ": ", total)
end

Boats held after month 1: 0.0
Boats held after month 2: 0.0
Boats held after month 3: 0.0
Boats held after month 4: 0.0
Boats held after month 5: 0.0
Boats held after month 6: 0.0
Boats held after month 7: 0.0
Boats held after month 8: 0.0
Boats held after month 9: 0.0
Boats held after month 10: 0.0
Boats held after month 11: 0.0
